In [1]:
#Load important Libraries
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
import shutil # to save it locally

In [2]:
HEADERS = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5','referer':'https://www.google.com/'})

In [3]:
data = pd.read_csv("products_data.csv")
data.head()

,name,link
0,"Kindle (10th Gen), 6"" Display with Built-in Li...",https://amazon.in/Staging-Product-Not-Retail-S...
1,Kindle Paperwhite (10th gen) - with Built-in L...,https://amazon.in/All-New-Kindle-Paperwhite-10...
2,"Kindle (10th Gen), 6"" Display with Built-in Li...",https://amazon.in/Staging-Product-Not-Retail-S...
3,Kindle Paperwhite (10th gen) -with Built-in Li...,https://amazon.in/All-New-Kindle-Paperwhite-10...
4,All-New Kindle Oasis (10th Gen) - Now with adj...,https://amazon.in/All-new-Kindle-Oasis/dp/B07L...


In [4]:
import re
# remove special characters
def remove_special_from_price(text):
    return re.sub('[^a-zA-Z0-9.]', '', text)
def remove_backsklashn(text):
    return re.sub('[\n]', '', text)
def remove_spaces(text):
    return text.strip()

In [5]:
products_info = pd.DataFrame(columns=['name','category','price','seller','return_exchange','warranty','no_of_images'])
products_info

,name,category,price,seller,return_exchange,warranty,no_of_images


In [6]:
for i,j in data.iterrows():
    count = 0
    link = j['link']
    page = requests.get(link, headers=HEADERS)
    if str(page)=="<Response [200]>":
        soup = BeautifulSoup(page.content)
        productStatus = soup.find('div',id = 'outOfStock')
        if type(productStatus)==None:
            productName = soup.find('span',id='productTitle')
            productName = productName.text.replace('\n','')
            productCategory = soup.find('span',class_='subnav-home')
            productCategory = remove_spaces(remove_backsklashn(productCategory.text))
            productPrice = soup.find('span',class_='priceBlockBuyingPriceString')
            productPrice = remove_special_from_price(productPrice.text)
            sellerName = soup.find('a',id='sellerProfileTriggerId')
            sellerName = sellerName.text
            tandc = soup.findAll('div',class_='icon-container')
            returnpolicy = remove_spaces(remove_backsklashn(tandc[0].text))
            warranty = remove_spaces(remove_backsklashn(tandc[2].text))
            #Finding First Image URL using BeautifulSoup
            firstImage = soup.find('img',id='landingImage')
            firstImage = firstImage['data-a-dynamic-image'].split('"')
            firstImage = firstImage[1]
            firstImage = firstImage.split('.')
            #Finding All Span Elements containing image URLs using BeautifulSoup
            images = soup.find_all('span',class_='a-button a-button-thumbnail a-button-toggle')
            #List for Saving Image Src Links
            imageLinks = [firstImage[0]+'.'+firstImage[1]+'.'+firstImage[2]+'.'+firstImage[4]]
            for e in images:
                f = e.find('img')
                #Removing image Elements that don't include image
                if f is not None:
                    #Splitting Image Sources to remove all images except jpg extension
                    b = f['src'].split('.')
                    #Removing image Elements with gif extension
                    if(b[-1]!='gif'):
                        #Appending final image URL to a list
                        imageLinks.append(b[0]+'.'+b[1]+'.'+b[2]+'.'+b[4])
            os.mkdir(productName)
            for element in imageLinks:
                filename=productName+'/'+str(count+1)+'.jpg'
                r = requests.get(element, stream = True, headers=HEADERS)
                if r.status_code == 200:
                    # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
                    r.raw.decode_content = True
                    # Open a local file with wb ( write binary ) permission.
                    with open(filename,'wb') as f:
                        shutil.copyfileobj(r.raw, f)
                    count=count+1
            print(str(count)+' Images Downloaded Successfully')
            products_info.loc[len(products_info.index)] = [productName,productCategory,productPrice,sellerName,returnpolicy,warranty,count]
        else:
            productName = soup.find('span',id='productTitle')
            productName = productName.text.replace('\n','')
            productCategory = soup.find('span',class_='nav-a-content')
            productCategory = remove_spaces(remove_backsklashn(productCategory.text))
            productNA = 'N.A.'
            products_info.loc[len(products_info.index)] = [productName,productCategory,productNA,productNA,productNA,productNA,productNA]


In [7]:
products_info.to_csv('products_info.csv')